In [1]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# tensorflow-privacy
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import seaborn as sns
import json
import os
import pandas as pd
from pandas import DataFrame
from numpy.random import choice, seed
from numpy import ndarray, concatenate, stack, array, round, zeros, arange
import numpy as np

import category_encoders as ce
from sklearn.preprocessing import RobustScaler

import pickle

from os import mkdir, path

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit

import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter
simplefilter('ignore', category=FutureWarning)
simplefilter('ignore', category=DeprecationWarning)

import scipy
print("scipy.__version__:  ", scipy.__version__)

import tensorflow_privacy.privacy
print("tensorflow_privacy.__version__:  ", tensorflow_privacy.__version__)

sns.set(style="whitegrid")


2024-12-11 05:29:54.772590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 05:29:55.497190: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


scipy.__version__:   1.11.1
tensorflow_privacy.__version__:   0.8.10


## 1. Read the TEST (non-Member) set (i.e., test set for the classifier model)

In [2]:
filehandler = open(f'/home/privacy-master/DataShare/Adult_test5k_capped',"rb")
dfTest=pickle.load(filehandler)
filehandler.close()

In [3]:
dfTest ["income"] = np.where(dfTest['income'] == '<=50K' , 0, 1)
dfTest["income"].value_counts()

income
0    3761
1    1239
Name: count, dtype: int64

## 2. Read the TRAIN sanitized sets (Member) (i.e., train dataset for the classifier model)

In [4]:
filehandler = open(f'/home/privacy-master/DataShare/Adult_k5_100.pkl',"rb")
dataFiles=pickle.load(filehandler)
filehandler.close()
len(dataFiles)

100

In [5]:
for i in range(len(dataFiles)):
    dataFiles[i]["income"] = np.where(dataFiles[i]['income'] == '<=50K' , 0, 1)

### Classifier begin

In [6]:
y_test = dfTest['income']
X_test = dfTest.drop(['income'], axis=1)

categorical = [col for col in X_test.columns if X_test[col].dtypes == 'O']
#print(categorical)
numerical = [col for col in X_test.columns if X_test[col].dtypes != 'O']
#print(numerical)

encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country'])
#TexasRawDF=TexasRawDF.drop(['RISK_MORTALITY'], axis=1)
encoder.fit(dfTest.drop(["income"], axis=1))
X_test = encoder.transform(X_test)

In [ ]:
######################################## Classifier Begin ###########################################
k_ = ["5"]
utility_score={}
MIA_score={}
for k in k_:#k = 10, 20, 35, 50
    utility_score[k]=[]
    MIA_score[k]=[]
    
    utility_score[k]=[]
    MIA_score[k]=[]

    
    for set_no in range(100):

        df=dataFiles[set_no]
        if (len(df)==0):
            continue
        count = df['income'].value_counts()


        y_train = df['income']
        X_train = df.drop(['income'], axis=1)
        X_train = encoder.transform(X_train)

        cols = X_train.columns
        scaler = RobustScaler()

        X_test = scaler.fit_transform(X_test)
        X_train = scaler.transform(X_train)

        X_train = pd.DataFrame(X_train, columns=[cols])
        X_test = pd.DataFrame(X_test, columns=[cols])

        # instantiate the classifier with n_estimators = 100
        rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)

        # fit the model to the training set
        rfc_100.fit(X_train, y_train)            

        # Predict on the test set results
        y_pred_100 = rfc_100.predict(X_test)

        # Check accuracy score 
        #print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))
        utility_score[k].append(accuracy_score(y_test, y_pred_100))
        


        
        INx_train_pred1 = rfc_100.predict_proba(X_train)
        OUTx_test_pred1 = rfc_100.predict_proba(X_test)

        
        INx_train_pred = np.zeros((len(X_train), 2))
        INx_train_pred[0:len(X_train), 0:INx_train_pred1.shape[1]]=INx_train_pred1
        OUTx_test_pred = np.zeros((len(X_test), 2))
        OUTx_test_pred[0:len(X_test), 0:OUTx_test_pred1.shape[1]] = OUTx_test_pred1

        probs_train = INx_train_pred
        probs_test = OUTx_test_pred
        #labels_train = y_train
        labels_train = np.zeros((len(INx_train_pred)))
        labels_train[0:len(INx_train_pred)] = y_train
        labels_train = labels_train.astype(int)
        #labels_test = y_test
        labels_test = np.zeros((len(OUTx_test_pred)))
        labels_test[0:len(OUTx_test_pred)] = y_test
        labels_test=labels_test.astype(int)

        # define what variables our attacker should have access to
        attack_input = AttackInputData(
          probs_train = probs_train,
          probs_test = probs_test,
          #loss_train = loss_train,
          #loss_test = loss_test,
          labels_train = labels_train,
          labels_test = labels_test,
          multilabel_data = False#True
        )

        # how should the data be sliced
        slicing_spec = SlicingSpec(
            entire_dataset = True,
            by_class = False,#not supported for multilabel
            by_percentiles = False,
            by_classification_correctness = False)#not supported for multilabel

        # define the type of attacker model that we want to use
        attack_types = [
            #AttackType.THRESHOLD_ATTACK,
            AttackType.RANDOM_FOREST#LOGISTIC_REGRESSION
        ]

        # run the attack
        attacks_result = mia.run_attacks(attack_input=attack_input,
                                         slicing_spec=slicing_spec,
                                         balance_attacker_training = False,
                                         attack_types=attack_types)

        #print(attacks_result.summary(by_slices=True))                        
        MIA_score[k].append(attacks_result.summary(by_slices=True))
        #print(attacks_result.summary(by_slices=True))

In [8]:
print("Classification Accuracy: ")
for k in k_:
    print(f"epoch = {k}")
    all_id_sum=0
    
    all_id_sum+= ((sum(utility_score[k]))/len(utility_score[k]))
    print("%.2f"% ((sum(utility_score[k]))/len(utility_score[k])))
    print(len(utility_score[k]))
    print("Average: %.2f"% (all_id_sum))

Classification Accuracy: 
epoch = 5
0.76
100
Average: 0.76


In [9]:
import re
print("MIA result: ")
for k in k_:
    print(f"k = {k}")
    all_id_AUCsum, all_id_ADVsum, all_id_PPVsum = 0, 0, 0
    
        
    temp_AUCsum, temp_ADVsum, temp_PPVsum = 0, 0, 0
    for str1 in MIA_score[k]:       
        result = re.findall(r"[-+]?\d*\.\d+|\d+", str1)
        temp_AUCsum += float(result[2])
        temp_ADVsum += float(result[5])
        temp_PPVsum += float(result[8])
    temp_AUCsum /= len(MIA_score[k])#100
    temp_ADVsum /= len(MIA_score[k])#100
    temp_PPVsum /= len(MIA_score[k])#100
    print("Average AUC: %.2f \tADV: %.2f \tPPV: %.2f"% (temp_AUCsum, temp_ADVsum, temp_PPVsum))

MIA result: 
k = 5
Average AUC: 0.84 	ADV: 0.54 	PPV: 0.99
